<a href="https://colab.research.google.com/github/shashanknagariya/neural-network/blob/main/neural_network_tutorial1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install nnfs

In [6]:
from nnfs.datasets import spiral_data
import numpy as np
import nnfs

In [7]:
x,y = spiral_data(100,3)

**DENSE LAYER CLASS**

In [12]:
nnfs.init()

class LayerDense:
  """
  A layer of neurons. This is called Layer Dense because
  every neuron in the previous layer connects to every
  neuron in the next layer.
  """

  def __init__(self,n_inputs,n_neurons):
    """
    class initialised with number of inputs and number of neurons
    weights are randomly initialised between -1 and 1
    biases are initialised to 0
    weights are nothing but the connections between the neurons
    biases are nothing but the biases of the neurons

    biases of neuron means how much the neuron is activated
    """
    self.weights = 0.10 * np.random.randn(n_inputs,n_neurons)
    self.biases = np.zeros((1,n_neurons))

  def forward(self,inputs):
    """
    forward pass of the layer.
    inputs are multiplied by the weights and added to the biases
    """
    self.output = np.dot(inputs,self.weights) + self.biases

In [13]:
dense1 = LayerDense(2,3)
dense1.forward(x)
print(dense1.output[:5])

[[ 0.          0.          0.        ]
 [ 0.00231199  0.00189689 -0.0009594 ]
 [ 0.00233242  0.00295563 -0.00267016]
 [ 0.00766013  0.00578583 -0.00236353]
 [ 0.01053444  0.00771121 -0.00284911]]


**ACTIVATION FUNCTION: RELU**

In [14]:
class ActivationReLU:
  def forward(self,inputs):
    """
    forward pass of the activation function.
    relu is applied to the inputs
    """
    self.output = np.maximum(0,inputs)

In [15]:
activation_1 = ActivationReLU()
activation_1.forward(dense1.output)
print(activation_1.output[:5])

[[0.         0.         0.        ]
 [0.00231199 0.00189689 0.        ]
 [0.00233242 0.00295563 0.        ]
 [0.00766013 0.00578583 0.        ]
 [0.01053444 0.00771121 0.        ]]


**ACTIVATION FUNCTION: SOFTMAX**

In [16]:
class ActivationSoftMax:
  def forward(self,inputs):
    """
    forward pass of the activation function.
    softmax is applied to the inputs
    where we subtract the maximum value from the inputs to avoid
    overflow
    and then take exponential of the result
    and then divide by the sum of the exponential of the result
    """
    exp_values = np.exp(inputs - np.max(inputs,axis=1,keepdims=True))
    probabilities = exp_values / np.sum(exp_values,axis=1,keepdims=True)
    self.output = probabilities


In [17]:
#make a forward pass through second dense layer
#it takes outputs of activation function of first layer as inputs
dense2 = LayerDense(3,3)
dense2.forward(activation_1.output)
print(dense2.output[:5])

[[ 0.0000000e+00  0.0000000e+00  0.0000000e+00]
 [ 2.9754589e-04 -7.6702145e-06  2.5199552e-04]
 [ 3.4295820e-04  7.2711578e-06  4.0575486e-04]
 [ 9.6534507e-04 -3.2600368e-05  7.6235103e-04]
 [ 1.3174859e-03 -4.8371196e-05  1.0126863e-03]]


In [18]:
#make a forward pass through activation function
#it takes the output of the second dense layer here
activation_2 = ActivationSoftMax()
activation_2.forward(dense2.output)
print(activation_2.output[:5])

[[0.33333334 0.33333334 0.33333334]
 [0.3333723  0.33327058 0.3333571 ]
 [0.33336365 0.33325174 0.33338457]
 [0.33346677 0.33313417 0.33339906]
 [0.33351895 0.33306375 0.3334173 ]]


**CALCULATING NETWORK ERROR WITH LOSS**

In [ ]:
# There are two ways to calculate network error with loss
# 1. CROSS ENTROPY LOSS BUILDING BLOCKS
# 2. IF DATA IS HOT ENCODED

**CROSS ENTROPY LOSS BUILDING BLOCKS**

In [ ]:
softmax_outpus = np.array([[]])

In [ ]:
# Generate a random softmax output for demonstration
# This is a placeholder and would normally come from the network's forward pass
random_softmax_output = np.array([[0.7, 0.2, 0.1],
                                 [0.1, 0.8, 0.1],
                                 [0.3, 0.3, 0.4]])

# Define the class targets (e.g., the true classes for each sample)
# These would typically be the 'y' values from your dataset
class_targets = np.array([0, 1, 2])

print("Random Softmax Output:")
print(random_softmax_output)
print("\nClass Targets:")
print(class_targets)

Now, let's see how to extract the relevant probabilities from the softmax output based on the `class_targets`. We want to get the probability that the model assigned to the *correct* class for each sample.

In [ ]:
# Extract the probabilities corresponding to the true class targets
# We use np.arange(len(random_softmax_output)) to get indices for each row
# and class_targets to get the column index for the correct class in each row
relevant_probabilities = random_softmax_output[np.arange(len(random_softmax_output)), class_targets]

print("Relevant Probabilities (probabilities of the true classes):")
print(relevant_probabilities)

These `relevant_probabilities` are the values we will use to calculate the loss, specifically the cross-entropy loss. A higher probability for the true class indicates a lower loss and better model performance.

In [19]:
class LOSS:
  def calculate(self, output, y):
    """
    calculate the data and regularization losses
    given model output and ground truth values
    here output is the softmax output and y is the class targets
    """
    sample_losses = self.forward(output, y)
    data_loss = np.mean(sample_losses)
    return data_loss

In [20]:
class LossCategoricalCrossEntropy(LOSS):
  def forward(self,y_pred,y_true):
    """
    forward pass of the categorical cross entropy loss
    """
    #number of samples in a batch
    sample_data = len(y_pred)
    #clip data to prevent dvision by 0
    # clip both side to not drag mean towards any value
    # what if y_pred becomes 0 then log will be inf and
    #if y_pred becomes 1 then log will be -
    y_pred_clipped = np.clip(y_pred,1e-7,1-1e-7)
    #probabilities for target values
    # only if categorical labels
    if len(y_true.shape) == 1:
      correct_confidences = y_pred_clipped[range(sample_data),y_true]
    elif len(y_true.shape) == 2:
      correct_confidences = np.sum(y_pred_clipped*y_true,axis=1)
    #losses
    negative_log_likelihoods = -np.log(correct_confidences)
    return negative_log_likelihoods


In [22]:
loss_function = LossCategoricalCrossEntropy()
loss = loss_function.calculate(activation_2.output,y)
print("Loss:",loss)

Loss: 1.0986664


In [23]:
# Define an Accuracy class
class Accuracy:
  def calculate(self, predictions, y):
    """
    Calculates the accuracy given predictions and true labels.
    """
    # Compare predictions to true labels and calculate accuracy
    accuracy = np.mean(predictions == y)
    return accuracy

# Calculate the predictions by taking the index of the highest probability
# if highest probability index is same as class target that means accuracy is
#perfect
predictions = np.argmax(activation_2.output, axis=1)

# Calculate and print the accuracy
accuracy_metric = Accuracy()
accuracy = accuracy_metric.calculate(predictions, y)
print("Accuracy:", accuracy)

Accuracy: 0.34
